<div class="well well-sm">  
    
This notebook is under development. For suggestions or feature request post your question at the [GenePattern help forum](https://groups.google.com/g/genepattern-help)
</div>

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Upload file (or type the name of a file that exists in the same directory as this notebook) and choose the separator (either tab-separated or comma-separated files are supperted for now).
    
Note: This project contains a text file called DESeq2_sample.txt which is the output of the DESeq2 module for reference.
</div>

In [1]:
import pandas as pd
@genepattern.build_ui(parameters={
    "filename": {
        "type": "file"
    },
    "sep": {
        "default": '\t',
        "type":"choice",
        "choices": {
            ',': ',',
            "Tab (\\t)": '\t',
        }
        },
    "output_var": {
        "name": "df",
        "default": "df",
        "hide": True,
    },
})
def read_file(filename,sep):
    df = pd.read_csv(filename,index_col=0,sep=sep)
    display(df.head())
    print('These are the columns you can choose from:\n')
    print('\n'.join(df.columns.values))
    return df

UIBuilder(function_import='nbtools.tool(id="read_file", origin="Notebook").function_or_method', name='read_fil…

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>  

    
From the columns of the DataFrame read above, select the ones you want as your x-axis and y-axis. You may choose to compute the log of those variables.
    
Note: For the sample data, the parameters are:  
    
- x-axis: log2 fold change (MLE): Factor EPZ6438 vs DMSO
- Transform x axis: False
- y-axis: Wald test p-value: Factor EPZ6438 vs DMSO
- Transform y axis: -log10(y_axis)
</div>

In [6]:

import urllib.request
import urllib.parse
import os
import genepattern


# Import packages
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

x_axis = 'ratio_1d'
y_axis = 'qval_1d'

compute_log10 = True

@genepattern.build_ui(parameters={
    "x_axis": {
        "type": "text",
        "description":"Full name of the column containing fold change",
    },
    "y_axis": {
        "type": "text",
        "description":"Full name of the column containing p-values",
    },
    "compute_log10": {
        "name":"Transform y_axis",
        "type":"choice",
        "default": True,
        "choices": {
            "-log10(y_axis)": True,
            "False": False,
        }
    },
    "compute_log2": {
        "name":"Transform x_axis",
        "default": False,
        "type":"choice",
        "choices": {
            "log2(x_axis)": True,
            "False": False,
        }
    }, 
    "output_var": {
        "hide": True,
    },
})
def make_plot(x_axis, compute_log2,y_axis,compute_log10):
    # Plot volcano plot
    fig = go.Figure()
    
    if (compute_log10 and compute_log2):
        y_flipped = f'-log10({y_axis})'
        df[y_flipped] = np.log10(df[y_axis]) * (-1)
        x_log = f'-log2({x_axis})'
        df[x_log] = np.log2(df[x_axis])
        
        fig_d1 = px.scatter(df, x=x_log, y=y_flipped)
    elif compute_log10:
        y_flipped = f'-log10({y_axis})'
        df[y_flipped] = np.log10(df[y_axis]) * (-1)
        fig_d1 = px.scatter(df, x=x_axis, y=y_flipped)
    elif compute_log2:
        x_log = f'-log2({x_axis})'
        df[x_log] = np.log2(df[x_axis])
        fig_d1 = px.scatter(df, x=x_log, y=y_axis)
    else:
        fig_d1 = px.scatter(df, x=x_axis, y=y_axis)


    # Plot volcano plot with text
    
    fig_d1.update_traces(textposition='top center')
#     fig_d1.update_layout(title_text='Volcano plot for seronegatives (day 1)')
    fig_d1.show()

UIBuilder(function_import='nbtools.tool(id="make_plot", origin="Notebook").function_or_method', name='make_plo…